# Run all

## リフレッシュトークンの設定

In [ ]:
# Googleドライブをマウント

from google.colab import drive
drive.mount('/content/drive')
refresh_token = ""
with open('/content/drive/MyDrive/drive_ws/jquantsapi-key') as f:
  refresh_token = f.readline().replace('\n', '')

Mounted at /content/drive


## ライブラリとデータ保存先の設定(5分)

In [ ]:
#!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12==24.2.* dask-cudf-cu12==24.2.* cuml-cu12==24.2.* cugraph-cu12==24.2.* cuspatial-cu12==24.2.* cuproj-cu12==24.2.* cuxfilter-cu12==24.2.* cucim-cu12==24.2.* pylibraft-cu12==24.2.* raft-dask-cu12==24.2.*

In [ ]:
# 必要なライブラリのインストールとインポート（20秒）
!pip install scikit-learn xgboost pandas numpy jquants-api-client

# J-Quants の株式分析チュートリアルからバックテスト用ライブラリを取得します。
# !wget -q https://raw.githubusercontent.com/JapanExchangeGroup/J-Quants-Tutorial/main/handson/Chapter03/backtest/backtest.py -O backtest.py

import os
import math
import pytz
import getpass
import io
import os
import pickle
from datetime import datetime, timedelta
from typing import List

import jquantsapi
import numpy as np
#%load_ext cudf.pandas
import pandas as pd
from matplotlib import pyplot as plt
# from backtest import Backtest
from dateutil import tz
from requests import HTTPError
from xgboost.sklearn import XGBRegressor

# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [ ]:
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

refresh_tokenは正常です。次の手順に進んでください。


In [ ]:
# 必要なコンフィグの定義
# Googleドライブをマウントするディレクトリ
GOOGLE_DRIVE_MOUNT_DIR_PATH = "/content/drive"
# データを保存しているGoogleドライブ上のディレクトリ
STORAGE_DIR_PATH = f"{GOOGLE_DRIVE_MOUNT_DIR_PATH}/MyDrive/drive_ws/marketdata/run"
RESULT_DIR_PATH = f"{GOOGLE_DRIVE_MOUNT_DIR_PATH}/MyDrive/drive_ws/result/"
# デバッグ中
# STORAGE_DIR_PATH = "/tmp/marketdata"
# モデルを保存しているGoogleドライブ上のディレクトリ
MODEL_DIR_PATH = f"{GOOGLE_DRIVE_MOUNT_DIR_PATH}/MyDrive/drive_ws/models"

# J-Quants API から取得するデータの期間
DAYS_IN_YESR = 365 # FIXME
#today = datetime(2024,2,27)
today = datetime.now(pytz.timezone('Asia/Tokyo'))

start_dt: datetime = today.replace(hour=23, minute=30, second=0, microsecond=0) - timedelta(days = math.floor(DAYS_IN_YESR * 0.5)) #昔の決算も入れる
end_dt: datetime = today.replace(hour=23, minute=30, second=0, microsecond=0)
start_dt_yyyymmdd = start_dt.strftime("%Y%m%d")
end_dt_yyyymmdd = end_dt.strftime("%Y%m%d")

# 各種CSVデータを保存するファイルパス
# 元データ
raw_stock_list_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_list_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
raw_stock_fins_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_fin_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
raw_stock_price_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_price_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
# 処理済みデータ
stock_list_csvfile_path = f"{STORAGE_DIR_PATH}/stock_list_uki_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
stock_fins_csvfile_path = f"{STORAGE_DIR_PATH}/stock_fin_uki_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
stock_price_csvfile_path = f"{STORAGE_DIR_PATH}/stock_price_uki_{start_dt_yyyymmdd}_{end_dt_yyyymmdd}.csv.gz"
stock_labels_csvfile_path = f"{GOOGLE_DRIVE_MOUNT_DIR_PATH}/MyDrive/drive_ws/marketdata/stock_labels_uki_model.csv.gz"
# 生成したモデルを保存するパス
model_path = f"{MODEL_DIR_PATH}/20220915_uki_model"

# トレーニング・検証データ期間
PURGING_DAYS = 31
TRAIN_DATA_DAYS = (3 * 365) - PURGING_DAYS  # FIXME: データ期間から割合で決定する
#train_start_dt: datetime = start_dt
#train_end_dt: datetime = start_dt + timedelta(days=TRAIN_DATA_DAYS)

# テスト・評価期間
#test_start_dt = train_end_dt + timedelta(days=PURGING_DAYS)
print("start_dt: ", start_dt)
print("end_dt: ", end_dt)
#print("train_start_dt: ", train_start_dt)
#print("train_end_dt: ", train_end_dt)
#print("test_start_dt: ", test_start_dt)

start_dt:  2023-08-30 23:30:00+09:00
end_dt:  2024-02-28 23:30:00+09:00


## データのダウンロードおよび保存
半年分なら3分

In [ ]:
cli = jquantsapi.Client(refresh_token=refresh_token)

#
# TODO: 日付取得してデータをマージする
#
# 銘柄一覧
if not os.path.isfile(raw_stock_list_csvfile_path):
    stock_list_load: pd.DataFrame = cli.get_list()
    stock_list_load.to_csv(raw_stock_list_csvfile_path, compression="gzip", index=False)
stock_list_load: pd.DataFrame = pd.read_csv(raw_stock_list_csvfile_path, dtype=str)
#data_type = stock_list_load["Code"].dtype
#print(f"The data type of 'Code': {data_type}")

# 株価情報
if not os.path.isfile(raw_stock_price_csvfile_path):
    stock_price_load: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
    stock_price_load.to_csv(raw_stock_price_csvfile_path, compression="gzip", index=False)
stock_price_load: pd.DataFrame = pd.read_csv(raw_stock_price_csvfile_path, dtype=str)

# 財務情報
if not os.path.isfile(raw_stock_fins_csvfile_path):
    stock_fin_load: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)
    stock_fin_load.to_csv(raw_stock_fins_csvfile_path, compression="gzip", index=False)
stock_fin_load: pd.DataFrame = pd.read_csv(raw_stock_fins_csvfile_path, dtype=str)

## データ処理
累積調整係数で2分かかる

In [ ]:
# 株価情報のobject型をdatetime64[ns]型に変換
stock_price_load["Date"] = pd.to_datetime(stock_price_load["Date"])

# 株価情報のいくつかがobject型になっているので数値型に変換
stock_price_load["AdjustmentOpen"] = stock_price_load["AdjustmentOpen"].astype(np.float64)
stock_price_load["AdjustmentHigh"] = stock_price_load["AdjustmentHigh"].astype(np.float64)
stock_price_load["AdjustmentLow"] = stock_price_load["AdjustmentLow"].astype(np.float64)
stock_price_load["AdjustmentClose"] = stock_price_load["AdjustmentClose"].astype(np.float64)
stock_price_load["AdjustmentFactor"] = stock_price_load["AdjustmentFactor"].astype(np.float64)
stock_price_load["TurnoverValue"] = stock_price_load["TurnoverValue"].astype(np.float64)

# 累積調整係数を作成します
def generate_cumulative_adjustment_factor(df):
   # 分割併合等の係数を適用日に変更
   df.loc[:, "AdjustmentFactor"] = df["AdjustmentFactor"].shift(-1).fillna(1.0)

   # 調整係数を作成するために逆順にソートする
   df = df.sort_values("Date", ascending=False)
   # 累積株価調整係数を作成
   df.loc[:, "EndOfDayQuote CumulativeAdjustmentFactor"] = 1 / df["AdjustmentFactor"].cumprod()
   # ソート順を昇順にする
   df = df.sort_values("Date")

   return df
# 累積調整係数を追加
stock_price_load = stock_price_load.sort_values(["Code", "Date"])
stock_price_load = stock_price_load.groupby("Code", group_keys=False).apply(generate_cumulative_adjustment_factor).reset_index(drop=True)

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"] = stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"].str[:-1]

In [ ]:
# 財務情報のいくつかがobject型になっているので数値型に変換
numeric_cols_fin = ['AverageNumberOfShares', 'BookValuePerShare', 'EarningsPerShare','Equity', 'EquityToAssetRatio',
                'ForecastDividendPerShare1stQuarter', 'ForecastDividendPerShare2ndQuarter', 'ForecastDividendPerShare3rdQuarter',
                'ForecastDividendPerShareAnnual', 'ForecastDividendPerShareFiscalYearEnd', 'ForecastEarningsPerShare', 'ForecastNetSales', 'ForecastOperatingProfit',
                'ForecastOrdinaryProfit', 'ForecastProfit', 'NetSales', 'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock',
                'OperatingProfit', 'OrdinaryProfit', 'Profit', 'ResultDividendPerShare1stQuarter','ResultDividendPerShare2ndQuarter','ResultDividendPerShare3rdQuarter',
                'ResultDividendPerShareAnnual','ResultDividendPerShareFiscalYearEnd','TotalAssets']
stock_fin_load[numeric_cols_fin] = stock_fin_load[numeric_cols_fin].apply(pd.to_numeric, errors='coerce', axis=1)

# 財務情報のobject型をdatetime64[ns]型に変換
stock_fin_load["DisclosedDate"] = pd.to_datetime(stock_fin_load["DisclosedDate"]) #開示
stock_fin_load["CurrentFiscalYearEndDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearEndDate"])  # 当事業年度終了日
stock_fin_load["CurrentFiscalYearStartDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearStartDate"])  # 当事業年度開始日
stock_fin_load["CurrentPeriodEndDate"] = pd.to_datetime(stock_fin_load["CurrentPeriodEndDate"]) # 当会計期間終了日

# 財務情報の値を調整します
stock_fin_load["Result_FinancialStatement FiscalYear"] = stock_fin_load["CurrentFiscalYearEndDate"].dt.strftime("%Y")

# 財務情報の同一日に複数レコードが存在することに対応します。
# ある銘柄について同一日に複数の開示が行われた場合レコードが重複します。
# ここでは簡易的に処理するために特定のTypeOfDocumentを削除した後に、開示時間順に並べて一番最後に発表された開示情報を採用しています。
stock_fin_load = stock_fin_load.loc[~stock_fin_load["TypeOfDocument"].isin(["ForecastRevision", "NumericalCorrection", "ForecastRevision_REIT"])]
stock_fin_load = stock_fin_load.sort_values(["DisclosedDate", "DisclosedTime"]).drop_duplicates(subset=["LocalCode", "DisclosedDate"], keep="last")

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_fin_load.loc[(stock_fin_load["LocalCode"].str.len() == 5) & (stock_fin_load["LocalCode"].str[-1] == "0"), "LocalCode"] = stock_fin_load.loc[(stock_fin_load["LocalCode"].str.len() == 5) & (stock_fin_load["LocalCode"].str[-1] == "0"), "LocalCode"].str[:-1]


In [ ]:
# 銘柄一覧
# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"] = stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"].str[:-1]

# 銘柄一覧と株価データから予測対象列を作成します

# prediction_target
# プライム(111)の500位～1000位の銘柄を選択
prime_targets = (stock_price_load.loc[
    (stock_price_load["Date"] >= "2022-01-01") &
    (stock_price_load["Code"].isin(stock_list_load.loc[stock_list_load["MarketCode"] == "0111", "Code"])),
    ["Code", "TurnoverValue"]
].groupby("Code").sum()).sort_values("TurnoverValue", ascending=False)[500:1000].index

# スタンダード(112)の500位～1000位の銘柄を選択
standard_targets = (stock_price_load.loc[
    (stock_price_load["Date"] >= "2022-01-01") &
    (stock_price_load["Code"].isin(stock_list_load.loc[stock_list_load["MarketCode"] == "0112", "Code"])),
    ["Code", "TurnoverValue"]
].groupby("Code").sum()).sort_values("TurnoverValue", ascending=False)[500:1000].index
# グロース株すべて
growth_targets = stock_list_load.loc[stock_list_load["MarketCode"] == "0113", "Code"]

prime_targets = pd.Index(prime_targets)
standard_targets = pd.Index(standard_targets)
growth_targets = pd.Index(growth_targets)
pred_targets = prime_targets.union(standard_targets).union(growth_targets)

target_list = stock_list_load.loc[stock_list_load["MarketCode"].isin(["0111", "0112", "0113"]), "Code"]


# prediction_target = プライム(111)、スタンダード(112)、グロース(113)の 2022-01-01 以降の売買代金上位2000銘柄
#target_list = stock_list_load.loc[stock_list_load["MarketCode"].isin(["0111", "0112", "0113"]), "Code"]
#pred_targets = (stock_price_load.loc[(stock_price_load["Date"] >= "2022-01-01") & (stock_price_load["Code"].isin(target_list)), ["Code", "TurnoverValue"]].groupby("Code").sum()).sort_values("TurnoverValue", ascending=False).head(2000).index

stock_list_load["prediction_target"] = stock_list_load["Code"].isin(pred_targets)
# universe_comp2 は、プライム、スタンダード、グロースのデータの最新日付時点の時価総額上位2000銘柄とします
latest_date = pd.to_datetime(sorted(stock_price_load["Date"].unique())[-1]).strftime("%Y-%m-%d")
df_atp = stock_price_load.loc[stock_price_load["Date"] == str(latest_date), ["Code", "AdjustmentClose"]]
df_atp = df_atp.loc[df_atp["Code"].isin(target_list)]
def _nos(df):
    df = df.sort_values(["DisclosedDate", "DisclosedTime"])
    df = df.ffill()
    df = df.tail(1)
    return df
df_nos = stock_fin_load.groupby("LocalCode")[["DisclosedDate", "DisclosedTime", "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]].apply(_nos)
df_nos = df_nos.reset_index()
df_mcap = pd.merge(df_atp, df_nos, left_on=["Code"], right_on=["LocalCode"], how="inner")
df_mcap["mcap"] = df_mcap["AdjustmentClose"] * df_mcap["NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]
universe_comp2 = sorted(df_mcap.sort_values("mcap").tail(2000)["Code"])
stock_list_load["universe_comp2"] = stock_list_load["Code"].isin(universe_comp2)


In [ ]:
# stock_list: データの互換性のための各種列名変換など
stock_list: pd.DataFrame = pd.DataFrame()
stock_list["Local Code"] = stock_list_load["Code"]

stock_list["Name (English)"] = stock_list_load["CompanyName"]  # 英語名の代わりに日本語名を設定
stock_list["Section/Products"] = stock_list_load["MarketCodeName"]
stock_list["33 Sector(Code)"] = stock_list_load["Sector33Code"]
stock_list["33 Sector(Name)"] = stock_list_load["Sector33CodeName"]
stock_list["17 Sector(Code)"] = stock_list_load["Sector17Code"]
stock_list["17 Sector(Name)"] = stock_list_load["Sector17CodeName"]
stock_list["Size (New Index Series)"] = stock_list_load["ScaleCategory"]
stock_list["prediction_target"] = stock_list_load["prediction_target"]
stock_list["universe_comp2"] = stock_list_load["universe_comp2"]


In [ ]:
# stock_price: データの互換性のための各種列名変換など
stock_price: pd.DataFrame = pd.DataFrame()
stock_price["Local Code"] = stock_price_load["Code"]
#stock_price["Date"] = stock_price_load["Date"]
stock_price["base_date"] = stock_price_load["Date"]
stock_price['EndOfDayQuote Date'] = stock_price_load["Date"]
stock_price["EndOfDayQuote Open"] = stock_price_load["AdjustmentOpen"]
stock_price["EndOfDayQuote High"] = stock_price_load["AdjustmentHigh"]
stock_price["EndOfDayQuote Low"] = stock_price_load["AdjustmentLow"]
stock_price["EndOfDayQuote Close"] = stock_price_load["AdjustmentClose"]
stock_price["EndOfDayQuote ExchangeOfficialClose"] = stock_price_load["AdjustmentClose"]
stock_price["EndOfDayQuote Volume"] = stock_price_load["AdjustmentVolume"]
stock_price["EndOfDayQuote CumulativeAdjustmentFactor"] = stock_price_load["EndOfDayQuote CumulativeAdjustmentFactor"]

stock_price.sort_values(["Local Code", "base_date"], inplace=True)
# ExchangeOfficialClose は欠損値がある場合は前日終値とします
stock_price["EndOfDayQuote ExchangeOfficialClose"] = stock_price.groupby(["Local Code"])["EndOfDayQuote ExchangeOfficialClose"].ffill()
# 前日終値の列を終値列から作成
stock_price["EndOfDayQuote PreviousClose"] = stock_price.groupby(["Local Code"])["EndOfDayQuote Close"].shift(1)
#stock_price.head()
stock_price.tail()

,Local Code,base_date,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose
518107,9997,2024-02-20,2024-02-20,619.0,619.0,614.0,616.0,616.0,218200.0,1.0,618.0
518108,9997,2024-02-21,2024-02-21,615.0,616.0,613.0,615.0,615.0,194900.0,1.0,616.0
518109,9997,2024-02-22,2024-02-22,615.0,618.0,612.0,613.0,613.0,246900.0,1.0,615.0
518110,9997,2024-02-26,2024-02-26,615.0,620.0,615.0,618.0,618.0,244600.0,1.0,613.0
518111,9997,2024-02-27,2024-02-27,618.0,623.0,617.0,619.0,619.0,265100.0,1.0,618.0


In [ ]:
# stock_financial: データの互換性のための各種列名変換など
stock_fin: pd.DataFrame = pd.DataFrame()
stock_fin["Local Code"] = stock_fin_load["LocalCode"]
stock_fin["Result_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentPeriodEndDate"].dt.strftime("%Y/%m")
stock_fin["Result_FinancialStatement TotalAssets"] = stock_fin_load["TotalAssets"] # 総資産
stock_fin["Result_FinancialStatement NetAssets"] = stock_fin_load["Equity"] # 純資産
stock_fin["Result_FinancialStatement NetSales"] = stock_fin_load["NetSales"] # 純売上高
stock_fin["Result_FinancialStatement OperatingIncome"] = stock_fin_load["OperatingProfit"] # 営業利益
stock_fin["Result_FinancialStatement OrdinaryIncome"] = stock_fin_load["OrdinaryProfit"]  # 経常利益
stock_fin["Result_FinancialStatement NetIncome"] = stock_fin_load["Profit"]  # 当期純利益
stock_fin["Result_FinancialStatement ReportType"] = stock_fin_load["TypeOfCurrentPeriod"] # {"1Q", "2Q", "3Q", "FY"}
stock_fin["Result_FinancialStatement FiscalYear"] = stock_fin_load["Result_FinancialStatement FiscalYear"]
stock_fin["base_date"] = stock_fin_load["DisclosedDate"] # 開示日
stock_fin["TypeOfDocument"] = stock_fin_load["TypeOfDocument"] # 書類種別
stock_fin["RetrospectiveRestatement"] = stock_fin_load["RetrospectiveRestatement"] #修正再表示フラグ
stock_fin["Forecast_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentFiscalYearEndDate"].dt.strftime("%Y/%m")
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement FiscalPeriodEnd"] = (
    stock_fin_load["CurrentFiscalYearEndDate"] + pd.Timedelta(365, unit="D")
).dt.strftime("%Y/%m")  # 本決算の場合は次の年度予想なので1年後の日付にします
stock_fin["Forecast_FinancialStatement ReportType"] = "FY"  #  予想は通期固定
stock_fin["Forecast_FinancialStatement NetSales"] = stock_fin_load["ForecastNetSales"]
stock_fin["Forecast_FinancialStatement OperatingIncome"] = stock_fin_load["ForecastOperatingProfit"]
stock_fin["Forecast_FinancialStatement NetIncome"] = stock_fin_load["ForecastProfit"]
stock_fin["Forecast_FinancialStatement OrdinaryIncome"] = stock_fin_load["ForecastOrdinaryProfit"]

#stock_fin.head()
stock_fin.tail()

,Local Code,Result_FinancialStatement FiscalPeriodEnd,Result_FinancialStatement TotalAssets,Result_FinancialStatement NetAssets,Result_FinancialStatement NetSales,Result_FinancialStatement OperatingIncome,Result_FinancialStatement OrdinaryIncome,Result_FinancialStatement NetIncome,Result_FinancialStatement ReportType,Result_FinancialStatement FiscalYear,base_date,TypeOfDocument,RetrospectiveRestatement,Forecast_FinancialStatement FiscalPeriodEnd,Forecast_FinancialStatement ReportType,Forecast_FinancialStatement NetSales,Forecast_FinancialStatement OperatingIncome,Forecast_FinancialStatement NetIncome,Forecast_FinancialStatement OrdinaryIncome
9491,4687,2024/03,NaN,NaN,NaN,NaN,NaN,NaN,FY,2024,2024-02-27,DividendForecastRevision,NaN,2025/03,FY,NaN,NaN,NaN,NaN
9492,4811,2023/12,3.948000e+09,1.822000e+09,4.440000e+09,577000000.0,563000000.0,424000000.0,FY,2023,2024-02-27,FYFinancialStatements_Consolidated_JP,false,2024/12,FY,NaN,NaN,NaN,NaN
9493,7901,2024/04,NaN,NaN,NaN,NaN,NaN,NaN,FY,2024,2024-02-27,EarnForecastRevision,NaN,2025/04,FY,NaN,NaN,NaN,NaN
9494,3666,2024/03,NaN,NaN,NaN,NaN,NaN,NaN,FY,2024,2024-02-27,DividendForecastRevision,NaN,2025/03,FY,NaN,NaN,NaN,NaN
9495,7363,2023/12,1.017000e+09,7.060000e+08,1.203000e+09,-31000000.0,-26000000.0,-100000000.0,FY,2023,2024-02-27,FYFinancialStatements_NonConsolidated_JP,false,2024/12,FY,NaN,NaN,NaN,NaN


## GoogleDriveへのCSV保存

In [ ]:
stock_list.to_csv(stock_list_csvfile_path, compression="gzip", index=False)
stock_price.to_csv(stock_price_csvfile_path, compression="gzip", index=False)
stock_fin.to_csv(stock_fins_csvfile_path, compression="gzip", index=False)

## データセットの読み込み

In [ ]:
# ノートブックの冪等性を高めるためこれらの変数はこのセル以外でいじらない
stock_list:pd.DataFrame = pd.read_csv(stock_list_csvfile_path, dtype={"Local Code": str})
stock_price:pd.DataFrame = pd.read_csv(stock_price_csvfile_path, dtype={"Local Code": str})
stock_fin:pd.DataFrame = pd.read_csv(stock_fins_csvfile_path, dtype={"Local Code": str})

stock_labels:pd.DataFrame = pd.read_csv(stock_labels_csvfile_path, dtype={"Local Code": str})
#data_type = stock_list["Local Code"].dtype
#print(f"The data type of 'Local Code': {data_type}")


# object型をdatetime64[ns]型に変換
stock_price["base_date"] = pd.to_datetime(stock_price["base_date"])
stock_price["EndOfDayQuote Date"] = pd.to_datetime(stock_price["EndOfDayQuote Date"])
stock_fin["base_date"] = pd.to_datetime(stock_fin["base_date"])
stock_fin["Result_FinancialStatement FiscalPeriodEnd"] = pd.to_datetime(stock_fin["Result_FinancialStatement FiscalPeriodEnd"])
stock_fin["Forecast_FinancialStatement FiscalPeriodEnd"] = pd.to_datetime(stock_fin["Forecast_FinancialStatement FiscalPeriodEnd"])
stock_labels["base_date"] = pd.to_datetime(stock_labels["base_date"])

# Unnamedな列を削除しておく
stock_price = stock_price.loc[:, ~stock_price.columns.str.contains('^Unnamed')]
stock_fin = stock_fin.loc[:, ~stock_fin.columns.str.contains('^Unnamed')]

#stock_fin.fillna(0)
print("stock_fin date range:", stock_fin['base_date'].min(), "to", stock_fin['base_date'].max())

stock_fin date range: 2023-08-30 00:00:00 to 2024-02-27 00:00:00


## モデルの関数定義

In [ ]:
stock_codes = sorted(stock_price["Local Code"].unique())


## 推論の実行
半年分のデータ = 13分

In [ ]:
# stock_priceを使ったテクニカル指標を各銘柄ごとに計算
def get_technical(stock_price:pd.DataFrame, code:int)->pd.DataFrame:
    technical_df = stock_price[stock_price["Local Code"] == code].copy()
    # 終値
    technical_df["close"] = technical_df["EndOfDayQuote Close"]
    # 騰落率
    technical_df["ror_1"] = technical_df["EndOfDayQuote Close"].pct_change(1)
    technical_df["ror_5"] = technical_df["EndOfDayQuote Close"].pct_change(5) # 5営業日前からの終値の変化率
    technical_df["ror_10"] = technical_df["EndOfDayQuote Close"].pct_change(10)
    technical_df["ror_20"] = technical_df["EndOfDayQuote Close"].pct_change(20)
    technical_df["ror_40"] = technical_df["EndOfDayQuote Close"].pct_change(40)
    technical_df["ror_60"] = technical_df["EndOfDayQuote Close"].pct_change(60)
    technical_df["ror_100"] = technical_df["EndOfDayQuote Close"].pct_change(100)

    # 売買代金
    technical_df["volume"] = technical_df["EndOfDayQuote Close"] * technical_df["EndOfDayQuote Volume"]
    technical_df = technical_df.replace([np.inf, -np.inf], np.nan)

    technical_df["vol_1"] = technical_df["volume"]
    technical_df["vol_5"] = technical_df["volume"].rolling(5).mean() # 5日移動平均
    technical_df["vol_10"] = technical_df["volume"].rolling(10).mean()
    technical_df["vol_20"] = technical_df["volume"].rolling(20).mean()
    technical_df["vol_40"] = technical_df["volume"].rolling(40).mean()
    technical_df["vol_60"] = technical_df["volume"].rolling(60).mean()
    technical_df["vol_100"] = technical_df["volume"].rolling(100).mean()
    technical_df["d_vol"] = technical_df["volume"] / technical_df["vol_20"]

    # レンジ (前日の終値に対して何%値動きしたか)
    technical_df["range"] = (
        technical_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote High"]].max(axis=1)
        - technical_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote Low"]].min(axis=1)
        ) / technical_df["EndOfDayQuote PreviousClose"]
    technical_df = technical_df.replace([np.inf, -np.inf], np.nan)

    # レンジの移動平均
    technical_df["atr_1"] = technical_df["range"]
    technical_df["atr_5"] = technical_df["range"].rolling(5).mean()
    technical_df["atr_10"] = technical_df["range"].rolling(10).mean()
    technical_df["atr_20"] = technical_df["range"].rolling(20).mean()
    technical_df["atr_40"] = technical_df["range"].rolling(40).mean()
    technical_df["atr_60"] = technical_df["range"].rolling(60).mean()
    technical_df["atr_100"] = technical_df["range"].rolling(100).mean()
    technical_df["d_atr"] = technical_df["range"] / technical_df["atr_20"]

    # ギャップ (前日の終値よりも当日の始値がどれだけ値動きしたか) のレンジ
    technical_df["gap_range"] = (np.abs(technical_df["EndOfDayQuote Open"] - technical_df["EndOfDayQuote PreviousClose"])) / technical_df[
        "EndOfDayQuote PreviousClose"]
    technical_df["g_atr_1"] = technical_df["gap_range"]
    technical_df["g_atr_5"] = technical_df["gap_range"].rolling(5).mean()
    technical_df["g_atr_10"] = technical_df["gap_range"].rolling(10).mean()
    technical_df["g_atr_20"] = technical_df["gap_range"].rolling(20).mean()
    technical_df["g_atr_40"] = technical_df["gap_range"].rolling(40).mean()
    technical_df["g_atr_60"] = technical_df["gap_range"].rolling(60).mean()
    technical_df["g_atr_100"] = technical_df["gap_range"].rolling(100).mean()

    # デイレンジ
    technical_df["day_range"] = (technical_df["EndOfDayQuote High"] - technical_df["EndOfDayQuote Low"]) / technical_df[
        "EndOfDayQuote PreviousClose"]
    technical_df["d_atr_1"] = technical_df["day_range"]
    technical_df["d_atr_5"] = technical_df["day_range"].rolling(5).mean()
    technical_df["d_atr_10"] = technical_df["day_range"].rolling(10).mean()
    technical_df["d_atr_20"] = technical_df["day_range"].rolling(20).mean()
    technical_df["d_atr_40"] = technical_df["day_range"].rolling(40).mean()
    technical_df["d_atr_60"] = technical_df["day_range"].rolling(60).mean()
    technical_df["d_atr_100"] = technical_df["day_range"].rolling(100).mean()

    # ヒゲレンジ: ((高値-低値) - (始値と終値の差))/終値
    technical_df["hig_range"] = ((technical_df["EndOfDayQuote High"] - technical_df["EndOfDayQuote Low"]) - np.abs(
        technical_df["EndOfDayQuote Open"] - technical_df["EndOfDayQuote Close"])) / technical_df["EndOfDayQuote PreviousClose"]
    technical_df["h_atr_1"] = technical_df["hig_range"]
    technical_df["h_atr_5"] = technical_df["hig_range"].rolling(5).mean()
    technical_df["h_atr_10"] = technical_df["hig_range"].rolling(10).mean()
    technical_df["h_atr_20"] = technical_df["hig_range"].rolling(20).mean()
    technical_df["h_atr_40"] = technical_df["hig_range"].rolling(40).mean()
    technical_df["h_atr_60"] = technical_df["hig_range"].rolling(60).mean()
    technical_df["h_atr_100"] = technical_df["hig_range"].rolling(100).mean()

    # ボラティリティ
    technical_df["vola_5"] = technical_df["ror_1"].rolling(5).std()
    technical_df["vola_10"] = technical_df["ror_1"].rolling(10).std()
    technical_df["vola_20"] = technical_df["ror_1"].rolling(20).std()
    technical_df["vola_40"] = technical_df["ror_1"].rolling(40).std()
    technical_df["vola_60"] = technical_df["ror_1"].rolling(60).std()
    technical_df["vola_100"] = technical_df["ror_1"].rolling(100).std()

    # HL(High-Low)バンド: 直近n日の高値（Hバンド）、安値（Lバンド）
    technical_df["hl_5"] = technical_df["EndOfDayQuote High"].rolling(5).max() - technical_df["EndOfDayQuote Low"].rolling(5).min()
    technical_df["hl_10"] = technical_df["EndOfDayQuote High"].rolling(10).max() - technical_df["EndOfDayQuote Low"].rolling(10).min()
    technical_df["hl_20"] = technical_df["EndOfDayQuote High"].rolling(20).max() - technical_df["EndOfDayQuote Low"].rolling(20).min()
    technical_df["hl_40"] = technical_df["EndOfDayQuote High"].rolling(40).max() - technical_df["EndOfDayQuote Low"].rolling(40).min()
    technical_df["hl_60"] = technical_df["EndOfDayQuote High"].rolling(60).max() - technical_df["EndOfDayQuote Low"].rolling(60).min()
    technical_df["hl_100"] = technical_df["EndOfDayQuote High"].rolling(100).max() - technical_df["EndOfDayQuote Low"].rolling(100).min()

    # マーケットインパクト
    technical_df["mi"] = technical_df["range"] / (technical_df["EndOfDayQuote Volume"] * technical_df["EndOfDayQuote Close"])
    technical_df = technical_df.replace([np.inf, -np.inf], np.nan) # "EndOfDayQuote Volume"=0の場合への考慮

    technical_df["mi_5"] = technical_df["mi"].rolling(5).mean()
    technical_df["mi_10"] = technical_df["mi"].rolling(10).mean()
    technical_df["mi_20"] = technical_df["mi"].rolling(20).mean()
    technical_df["mi_40"] = technical_df["mi"].rolling(40).mean()
    technical_df["mi_60"] = technical_df["mi"].rolling(60).mean()
    technical_df["mi_100"] = technical_df["mi"].rolling(100).mean()

    feat = ["EndOfDayQuote Date", "Local Code", "close",
            "ror_1", "ror_5", "ror_10", "ror_20", "ror_40", "ror_60", "ror_100",
            "vol_1", "vol_5", "vol_10", "vol_20", "vol_40", "vol_60", "vol_100", "d_vol",
            "atr_1", "atr_5", "atr_10", "atr_20", "atr_40", "atr_60", "atr_100", "d_atr",
            "g_atr_1", "g_atr_5", "g_atr_10", "g_atr_20", "g_atr_40", "g_atr_60", "g_atr_100",
            "d_atr_1", "d_atr_5", "d_atr_10", "d_atr_20", "d_atr_40", "d_atr_60", "d_atr_100",
            "h_atr_1", "h_atr_5", "h_atr_10", "h_atr_20", "h_atr_40", "h_atr_60", "h_atr_100",
            "vola_5", "vola_10", "vola_20", "vola_40", "vola_60", "vola_100",
            "hl_5", "hl_10", "hl_20", "hl_40", "hl_60", "hl_100",
            "mi_5", "mi_10", "mi_20", "mi_40", "mi_60", "mi_100"]
    technical_df = technical_df[feat]
    technical_df.columns = ["datetime", "code", "close",
                      "ror_1", "ror_5", "ror_10", "ror_20", "ror_40", "ror_60", "ror_100",
                      "vol_1", "vol_5", "vol_10", "vol_20", "vol_40", "vol_60", "vol_100", "d_vol",
                      "atr_1", "atr_5", "atr_10", "atr_20", "atr_40", "atr_60", "atr_100", "d_atr",
                      "g_atr_1", "g_atr_5", "g_atr_10", "g_atr_20", "g_atr_40", "g_atr_60", "g_atr_100",
                      "d_atr_1", "d_atr_5", "d_atr_10", "d_atr_20", "d_atr_40", "d_atr_60", "d_atr_100",
                      "h_atr_1", "h_atr_5", "h_atr_10", "h_atr_20", "h_atr_40", "h_atr_60", "h_atr_100",
                      "vola_5", "vola_10", "vola_20", "vola_40", "vola_60", "vola_100",
                      "hl_5", "hl_10", "hl_20", "hl_40", "hl_60", "hl_100",
                      "mi_5", "mi_10", "mi_20", "mi_40", "mi_60", "mi_100"]
    technical_df["datetime"] = pd.to_datetime(technical_df["datetime"])
    technical_df = technical_df.set_index(["datetime", "code"])
    return technical_df

# example
# get_technical(stock_price, example_codes[0]).tail()



# stock_finを使った指標を各銘柄ごとに計算
def get_financial(stock_fin:pd.DataFrame, code:int)->pd.DataFrame:

    fin_df = stock_fin[stock_fin["Local Code"] == code].copy()

    #print("get_financial() fin_df date range:", code, fin_df['base_date'].min(), "to", fin_df['base_date'].max())
    #print("get_financial() stock_fin date range:", code, stock_fin['base_date'].min(), "to", stock_fin['base_date'].max())

    # TypeOfDocumentの値によってはTotalAssetsなどの値がNaNになっているのでffill
    fin_df = fin_df.ffill()

    # --- 本決算／中間決算フラグ、修正開示フラグ、事後修正有無フラグ ---
    fin_df["annual"] = 0 # 0: 中間決算, 1:本決算
    fin_df["revision"] = 0 # 1: 修正再表示
    # FYFinancialStatements*** は本決算
    fin_df.loc[fin_df["TypeOfDocument"].isin(["FYFinancialStatements_Consolidated_JP", "FYFinancialStatements_Consolidated_US", "FYFinancialStatements_Consolidated_IFRS"]), "annual"] = 1
    fin_df.loc[fin_df["RetrospectiveRestatement"]==True, "revision"] = 1
    feat1 = ["annual", "revision"]

    # --- 原系列 ---
    # (memo: 前期と当期を比較すると季節的な変動が含まれ、経済的な変動を評価しづらくなる。この季節的な変動を取り除く前のものを原系列と呼ぶ)

    # --- r_sales ---
    # pre_result_period_end: 前回の情報開示日時点における事業年度終了日
    fin_df["pre_result_period_end"] = fin_df["Result_FinancialStatement FiscalPeriodEnd"].shift(1)
    fin_df["r_sales"] = np.nan

    # 前の行から会計年度が変わったけどReportTypeが1Qでない = 年度の途中からのデータを見ており一番最初のレコードが2Qとか3Qのものだったとき?よくわかってない
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] != "1Q")), "r_sales"] = fin_df[
        "Result_FinancialStatement NetSales"].diff(1)
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] == "1Q")), "r_sales"] = fin_df[
        "Result_FinancialStatement NetSales"]
    fin_df["r_sales"] = fin_df["r_sales"].ffill()

    # --- r_ope_income ---
    fin_df["r_ope_income"] = np.nan
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] != "1Q")), "r_ope_income"] = fin_df[
        "Result_FinancialStatement OperatingIncome"].diff(1)
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] == "1Q")), "r_ope_income"] = fin_df[
        "Result_FinancialStatement OperatingIncome"]
    fin_df["r_ope_income"] = fin_df["r_ope_income"].ffill()

    # --- r_ord_income ---
    fin_df["r_ord_income"] = np.nan
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] != "1Q")), "r_ord_income"] = fin_df[
        "Result_FinancialStatement OrdinaryIncome"].diff(1)
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] == "1Q")), "r_ord_income"] = fin_df[
        "Result_FinancialStatement OrdinaryIncome"]
    fin_df["r_ord_income"] = fin_df["r_ord_income"].ffill()

    # --- r_net_income ---
    fin_df["r_net_income"] = np.nan
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] != "1Q")), "r_net_income"] = fin_df[
        "Result_FinancialStatement NetIncome"].diff(1)
    fin_df.loc[((fin_df["Result_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_result_period_end"]) & (
            fin_df["Result_FinancialStatement ReportType"] == "1Q")), "r_net_income"] = fin_df[
        "Result_FinancialStatement NetIncome"]
    fin_df["r_net_income"] = fin_df["r_net_income"].ffill()

    # --- pre_forcast_period_end ---
    fin_df["pre_forecast_period_end"] = fin_df["Forecast_FinancialStatement FiscalPeriodEnd"].shift(1)

    # --- f_sales ---
    fin_df["f_sales"] = np.nan
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] != "1Q")), "f_sales"] = fin_df[
        "Forecast_FinancialStatement NetSales"].diff(1)
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] == "1Q")), "f_sales"] = fin_df[
        "Forecast_FinancialStatement NetSales"]
    fin_df["f_sales"] = fin_df["f_sales"].ffill()

    # --- f_ope_income ---
    fin_df["f_ope_income"] = np.nan
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] != "1Q")), "f_ope_income"] = fin_df[
        "Forecast_FinancialStatement OperatingIncome"].diff(1)
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] == "1Q")), "f_ope_income"] = fin_df[
        "Forecast_FinancialStatement OperatingIncome"]
    fin_df["f_ope_income"] = fin_df["f_ope_income"].ffill()

    # --- f_ord_income ---
    fin_df["f_ord_income"] = np.nan
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] != "1Q")), "f_ord_income"] = fin_df[
        "Forecast_FinancialStatement OrdinaryIncome"].diff(1)
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] == "1Q")), "f_ord_income"] = fin_df[
        "Forecast_FinancialStatement OrdinaryIncome"]
    fin_df["f_ord_income"] = fin_df["f_ord_income"].ffill()

    # --- f_net_income ---
    fin_df["f_net_income"] = np.nan
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] != "1Q")), "f_net_income"] = fin_df[
        "Forecast_FinancialStatement NetIncome"].diff(1)
    fin_df.loc[((fin_df["Forecast_FinancialStatement FiscalPeriodEnd"] != fin_df["pre_forecast_period_end"]) & (
            fin_df["Forecast_FinancialStatement ReportType"] == "1Q")), "f_net_income"] = fin_df[
        "Forecast_FinancialStatement NetIncome"]
    fin_df["f_net_income"] = fin_df["f_net_income"].ffill()

    # --------------------
    fin_df["r_expense1"] = fin_df["r_sales"] - fin_df["r_ope_income"]
    fin_df["r_expense2"] = fin_df["r_ope_income"] - fin_df["r_ord_income"]
    fin_df["r_expense3"] = fin_df["r_ord_income"] - fin_df["r_net_income"]

    fin_df["f_expense1"] = fin_df["f_sales"] - fin_df["f_ope_income"]
    fin_df["f_expense2"] = fin_df["f_ope_income"] - fin_df["f_ord_income"]
    fin_df["f_expense3"] = fin_df["f_ord_income"] - fin_df["f_net_income"]

    fin_df["r_assets"] = fin_df["Result_FinancialStatement TotalAssets"]
    fin_df["r_equity"] = fin_df["Result_FinancialStatement NetAssets"]

    # 現在 J-Quants APIからは取れなさそう
    # fin_df["operating_cf"] = fin_df["Result_FinancialStatement CashFlowsFromOperatingActivities"]
    # fin_df["financial_cf"] = fin_df["Result_FinancialStatement CashFlowsFromFinancingActivities"]
    # fin_df["investing_cf"] = fin_df["Result_FinancialStatement CashFlowsFromInvestingActivities"]

    feat2 = ["r_sales", "r_ope_income", "r_ord_income", "r_net_income", "f_sales", "f_ope_income", "f_ord_income",
            "f_net_income",
            "r_expense1", "r_expense2", "r_expense3", "f_expense1", "f_expense2", "f_expense3",
            "r_assets", "r_equity",] #"operating_cf", "financial_cf", "investing_cf"]


    # --- 複合指標　原系列 ---
    # ------ 純利益系 ------
    fin_df["r_pm1"] = fin_df["Result_FinancialStatement NetIncome"] / fin_df["Result_FinancialStatement NetSales"]
    fin_df["r_roe1"] = fin_df["Result_FinancialStatement NetIncome"] / fin_df["Result_FinancialStatement NetAssets"]
    fin_df["r_roa1"] = fin_df["Result_FinancialStatement NetIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    fin_df["f_pm1"] = fin_df["Forecast_FinancialStatement NetIncome"] / fin_df[
        "Forecast_FinancialStatement NetSales"]
    fin_df["f_roe1"] = fin_df["Forecast_FinancialStatement NetIncome"] / fin_df[
        "Result_FinancialStatement NetAssets"]
    fin_df["f_roa1"] = fin_df["Forecast_FinancialStatement NetIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    # 経常利益系
    fin_df["r_pm2"] = fin_df["Result_FinancialStatement OrdinaryIncome"] / fin_df[
        "Result_FinancialStatement NetSales"]
    fin_df["r_roe2"] = fin_df["Result_FinancialStatement OrdinaryIncome"] / fin_df[
        "Result_FinancialStatement NetAssets"]
    fin_df["r_roa2"] = fin_df["Result_FinancialStatement OrdinaryIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    fin_df["f_pm2"] = fin_df["Forecast_FinancialStatement OrdinaryIncome"] / fin_df[
        "Forecast_FinancialStatement NetSales"]
    fin_df["f_roe2"] = fin_df["Forecast_FinancialStatement OrdinaryIncome"] / fin_df[
        "Result_FinancialStatement NetAssets"]
    fin_df["f_roa2"] = fin_df["Forecast_FinancialStatement OrdinaryIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    # 営業利益系
    fin_df["r_pm3"] = fin_df["Result_FinancialStatement OperatingIncome"] / fin_df[
        "Result_FinancialStatement NetSales"]
    fin_df["r_roe3"] = fin_df["Result_FinancialStatement OperatingIncome"] / fin_df[
        "Result_FinancialStatement NetAssets"]
    fin_df["r_roa3"] = fin_df["Result_FinancialStatement OperatingIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    fin_df["f_pm3"] = fin_df["Forecast_FinancialStatement OperatingIncome"] / fin_df[
        "Forecast_FinancialStatement NetSales"]
    fin_df["f_roe3"] = fin_df["Forecast_FinancialStatement OperatingIncome"] / fin_df[
        "Result_FinancialStatement NetAssets"]
    fin_df["f_roa3"] = fin_df["Forecast_FinancialStatement OperatingIncome"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    # コスト
    fin_df["r_cost1"] = ((fin_df["Result_FinancialStatement NetSales"] - fin_df[
        "Result_FinancialStatement OperatingIncome"]) / fin_df["Result_FinancialStatement NetSales"])
    fin_df["r_cost2"] = ((fin_df["Result_FinancialStatement OperatingIncome"] - fin_df[
        "Result_FinancialStatement OrdinaryIncome"]) / fin_df["Result_FinancialStatement NetSales"])
    fin_df["r_cost3"] = ((fin_df["Result_FinancialStatement OrdinaryIncome"] - fin_df[
        "Result_FinancialStatement NetIncome"]) / fin_df["Result_FinancialStatement NetSales"])

    fin_df["f_cost1"] = ((fin_df["Forecast_FinancialStatement NetSales"] - fin_df[
        "Forecast_FinancialStatement OperatingIncome"]) / fin_df["Forecast_FinancialStatement NetSales"])
    fin_df["f_cost2"] = ((fin_df["Forecast_FinancialStatement OperatingIncome"] - fin_df[
        "Forecast_FinancialStatement OrdinaryIncome"]) / fin_df["Forecast_FinancialStatement NetSales"])
    fin_df["f_cost3"] = ((fin_df["Forecast_FinancialStatement OrdinaryIncome"] - fin_df[
        "Forecast_FinancialStatement NetIncome"]) / fin_df["Forecast_FinancialStatement NetSales"])

    # 売上高回転率
    fin_df["r_turn"] = fin_df["Result_FinancialStatement NetSales"] / fin_df[
        "Result_FinancialStatement TotalAssets"]
    fin_df["f_turn"] = fin_df["Forecast_FinancialStatement NetSales"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    # 財務健全性
    fin_df["equity_ratio"] = fin_df["Result_FinancialStatement NetAssets"] / fin_df[
        "Result_FinancialStatement TotalAssets"]

    # 総資本キャッシュフロー比率 --- 現在J-Quants APIからは取得できなさそう
    # fin_df["o_cf_ratio"] = (fin_df["Result_FinancialStatement CashFlowsFromOperatingActivities"] / fin_df[
    #     "Result_FinancialStatement TotalAssets"])
    # fin_df["f_cf_ratio"] = (fin_df["Result_FinancialStatement CashFlowsFromFinancingActivities"] / fin_df[
    #     "Result_FinancialStatement TotalAssets"])
    # fin_df["i_cf_ratio"] = (fin_df["Result_FinancialStatement CashFlowsFromInvestingActivities"] / fin_df[
    #     "Result_FinancialStatement TotalAssets"])

    feat3 = ["r_pm1", "r_roe1", "r_roa1", "f_pm1", "f_roe1", "f_roa1",
             "r_pm2", "r_roe2", "r_roa2", "f_pm2", "f_roe2", "f_roa2",
             "r_pm3", "r_roe3", "r_roa3", "f_pm3", "f_roe3", "f_roa3",
             "r_cost1", "r_cost2", "r_cost3", "f_cost1", "f_cost2", "f_cost3",
             "r_turn", "f_turn", "equity_ratio", ] # "o_cf_ratio", "f_cf_ratio", "i_cf_ratio"]

    # Inf値をNan値化
    fin_df = fin_df.replace([np.inf, -np.inf], np.nan)

    # 差分系列
    d_feat2 = []

    for f in feat2:
        fin_df["d_" + f] = fin_df[f].diff(1)
        d_feat2.append("d_" + f)

    d_feat3 = []
    for f in feat3:
        fin_df["d_" + f] = fin_df[f].diff(1)
        d_feat3.append("d_" + f)

    d_feat4 = ["m_sales", "m_ope_income", "m_ord_income", "m_net_income", "m_expense1", "m_expense2", "m_expense3",
               "m_pm1", "m_pm2", "m_pm3", "m_roe1", "m_roe2", "m_roe3", "m_roa1", "m_roa2", "m_roa3",
               "m_cost1", "m_cost2", "m_cost3"]

    fin_df["m_sales"] = fin_df["r_sales"] - fin_df["f_sales"].shift(1)
    fin_df["m_ope_income"] = fin_df["r_ope_income"] - fin_df["f_ope_income"].shift(1)
    fin_df["m_ord_income"] = fin_df["r_ord_income"] - fin_df["f_ord_income"].shift(1)
    fin_df["m_net_income"] = fin_df["r_net_income"] - fin_df["f_net_income"].shift(1)
    fin_df["m_expense1"] = fin_df["r_expense1"] - fin_df["f_expense1"].shift(1)
    fin_df["m_expense2"] = fin_df["r_expense2"] - fin_df["f_expense2"].shift(1)
    fin_df["m_expense3"] = fin_df["r_expense3"] - fin_df["f_expense3"].shift(1)

    fin_df["m_pm1"] = fin_df["r_pm1"] - fin_df["f_pm1"].shift(1)
    fin_df["m_pm2"] = fin_df["r_pm2"] - fin_df["f_pm2"].shift(1)
    fin_df["m_pm3"] = fin_df["r_pm3"] - fin_df["f_pm3"].shift(1)
    fin_df["m_roe1"] = fin_df["r_roe1"] - fin_df["f_roe1"].shift(1)
    fin_df["m_roe2"] = fin_df["r_roe2"] - fin_df["f_roe2"].shift(1)
    fin_df["m_roe3"] = fin_df["r_roe3"] - fin_df["f_roe3"].shift(1)
    fin_df["m_roa1"] = fin_df["r_roa1"] - fin_df["f_roa1"].shift(1)
    fin_df["m_roa2"] = fin_df["r_roa2"] - fin_df["f_roa2"].shift(1)
    fin_df["m_roa3"] = fin_df["r_roa3"] - fin_df["f_roa3"].shift(1)
    fin_df["m_cost1"] = fin_df["r_cost1"] - fin_df["f_cost1"].shift(1)
    fin_df["m_cost2"] = fin_df["r_cost2"] - fin_df["f_cost2"].shift(1)
    fin_df["m_cost3"] = fin_df["r_cost3"] - fin_df["f_cost3"].shift(1)

    feat = ["base_date", "Local Code"]
    feat.extend(feat1)
    feat.extend(feat2)
    feat.extend(feat3)
    feat.extend(d_feat2)
    feat.extend(d_feat3)
    feat.extend(d_feat4)

    col_names = ["datetime", "code"]
    col_names.extend(feat1)
    col_names.extend(feat2)
    col_names.extend(feat3)
    col_names.extend(d_feat2)
    col_names.extend(d_feat3)
    col_names.extend(d_feat4)

    fin_df = fin_df[feat]
    fin_df.columns = col_names
    fin_df["datetime"] = pd.to_datetime(fin_df["datetime"])
    fin_df = fin_df.set_index(["datetime", "code"])
    return fin_df

# example
#get_financial(stock_fin, example_codes[1]).tail()



def get_df_merge(stock_price:pd.DataFrame, stock_fin:pd.DataFrame, stock_labels:pd.DataFrame, stock_codes, train:bool=True):
    df_technical = []
    for code in stock_codes:
        print("technical code: ", code)
        df_technical.append(get_technical(stock_price, code))
    df_technical = pd.concat(df_technical)

    df_financial = []
    for code in stock_codes:
        print("financial code: ", code)
        df_financial.append(get_financial(stock_fin, code))

    #print("stock_codes len: ", len(stock_codes))
    df_financial = pd.concat(df_financial)

    # TypeOfDocument=NumerialCorrectionなどの場合において、　Disclosed Dateが同じのドキュメントが同一銘柄で存在したりする。
    # datetime-codeをuniqueなindexとしたいので一旦そういった重複日のドキュメントは排除する
    df_financial = df_financial[~df_financial.index.duplicated(keep='first')]

    # df_financial DataFrameの日付範囲を確認
    #print("df_financial date range:", df_financial.index.get_level_values('datetime').min(), "to", df_financial.index.get_level_values('datetime').max())
    # df_technical DataFrameの日付範囲を確認
    #print("df_technical date range:", df_technical.index.get_level_values('datetime').min(), "to", df_technical.index.get_level_values('datetime').max())


    if train:
        df_label = stock_labels.copy()
        feat = ["base_date", "Local Code", "label_high_20", "label_low_20"]
        df_label = df_label[feat]
        df_label.columns = ["datetime", "code", "label_high_20", "label_low_20"]

        df_label["datetime"] = pd.to_datetime(df_label["datetime"])
        df_label = df_label.set_index(["datetime", "code"])

        df_merge = pd.concat([df_financial,
                              df_technical[df_technical.index.isin(df_financial.index)],
                              df_label[df_label.index.isin(df_financial.index)]
                              ], axis=1)
    else:
        df_merge = pd.concat([df_financial,
                              df_technical[df_technical.index.isin(df_financial.index)],
                              ], axis=1)

    df_merge = df_merge.reset_index()
    return df_merge

def get_df_for_ml(stock_price:pd.DataFrame, stock_fin:pd.DataFrame, stock_labels:pd.DataFrame, stock_codes, train=True):
    df_merge = get_df_merge(stock_price, stock_fin, stock_labels, stock_codes, train=train)
    df_merge = df_merge.replace([np.inf, -np.inf], np.nan)
    df_merge = df_merge.fillna(0)
    return df_merge

def get_recent_statements(df, dt, latest_n=100):
    for cnt in range(100):
        tmp_df = df[(df["datetime"] > dt - timedelta(cnt)) & (df["datetime"] <= dt)].copy()
        if len(tmp_df) >= latest_n:
            break
    tmp_df["datetime"] = dt
    return tmp_df

def get_df_for_predict(stock_price:pd.DataFrame, stock_fin:pd.DataFrame, stock_labels:pd.DataFrame, stock_codes, train=False, latest_n=100):
    df_merge = get_df_merge(stock_price, stock_fin, stock_labels, stock_codes, train=train)
    df_merge = df_merge.replace([np.inf, -np.inf], np.nan)
    df_merge = df_merge.fillna(0)
    tmp = df_merge[df_merge["datetime"]>="2024-01-01"]
    print("df tmp: ", tmp.tail())

    #dt_list = pd.to_datetime(sorted(df_merge["datetime"].unique()))
    #dt_list = pd.Series(dt_list)
    # # FRIDAY = 4
    # dt_list = dt_list[dt_list.dt.dayofweek == FRIDAY]
    #tmp_dfs = []
    #for dts in dt_list:
    #    tmp = get_recent_statements(df_merge, dts, latest_n)
    #    tmp_dfs.append(tmp)
    #df_merge = pd.concat(tmp_dfs)
    #df_merge = df_merge[df_merge["datetime"]>="2020-01-01"]
    return df_merge

def get_model(model_path):
    models = {}
    labels = ["model_h_final", "model_l_final"]
    for label in labels:
        m = os.path.join(model_path, f"my_model_{label}.pkl")
        with open(m, "rb") as f:
            models[label] = pickle.load(f)
    return models["model_h_final"], models["model_l_final"]

def get_predict(df_for_ml, models_h, models_l):
    tmp_df = df_for_ml.copy()

    x_feats = [f for f in tmp_df.columns if f not in ["datetime", "code", "label_high_20", "label_low_20"]]

    tmp_df["pred_high_20"] = models_h.predict(tmp_df[x_feats])
    tmp_df["pred_low_20"] = models_l.predict(tmp_df[x_feats])

    tmp_df = tmp_df.set_index(["datetime"])
    cols = ["code", "pred_high_20", "pred_low_20"]
    tmp_df = tmp_df[cols]

    return tmp_df

def predict(stock_price:pd.DataFrame, stock_fin:pd.DataFrame, stock_labels:pd.DataFrame, stock_codes, latest_n=100):
    # 特徴量を作成
    # df_for_ml = get_df_for_ml(stock_price, stock_fin, stock_labels, stock_codes, train=False)
    df_for_ml = get_df_for_predict(stock_price, stock_fin, stock_labels, stock_codes, train=False, latest_n=latest_n)
    print(df_for_ml.tail())

    # 予測
    models_h, models_l = get_model(model_path)
    df = get_predict(df_for_ml, models_h, models_l)
    df = df.sort_values(["datetime", "pred_high_20", "code"], ascending=True)

    return df

result:pd.DataFrame = predict(stock_price, stock_fin, stock_labels, stock_codes, latest_n = 200)


## 予測の確認

In [ ]:
# 予測を出力します
#result.tail(30)
#result
end = datetime(2024, 2, 28)
#end = today
start = end - timedelta(days=1) #N日間で強い銘柄を列挙する
print(f"start = {start}, end = {end}")
filtered_result = result[ (start <= result.index) & (result.index <= end) ].sort_values(by='pred_high_20', ascending=False)
filtered_result

start = 2024-02-27 00:00:00, end = 2024-02-28 00:00:00


,code,pred_high_20,pred_low_20
datetime,,,
2024-02-27,6862,0.138060,-0.141800
2024-02-27,2708,0.120025,-0.151793
2024-02-27,7901,0.113703,-0.138726
2024-02-27,4811,0.111123,-0.178071
2024-02-27,7363,0.097370,-0.150267
2024-02-27,4687,0.089388,-0.097850
2024-02-27,5571,0.079389,-0.079000
2024-02-27,3666,0.072209,-0.111321
2024-02-27,6508,0.064338,-0.067911


In [131]:
#
# TODO: いつ上場したか, PER, を取得する
#

import urllib.parse
prompt1 = "でインターネットで検索して、その会社のホームページ、最新のIRの情報で株価がどうなりそうか予測してみて"
prompt2 = "でインターネットで検索して、その会社の「ビジネスモデル」と「この会社の利益が入る仕組み」をそれぞれ列挙して。"


# Google検索URLを生成する関数
def create_google_search_url(company_name, company_code):
    base_url = "https://www.google.com/search?q="
    query_string = f"{company_name} {company_code}"  # 会社名と会社コードをスペースで区切って組み合わせ
    encoded_query = urllib.parse.quote(query_string)  # 組み合わせた文字列をURLエンコード
    return base_url + encoded_query

def create_bing_search_url(company_name, company_code, prompt):
    base_url = "https://www.bing.com/search?q="
    query_string = f"{company_name} {company_code} " + prompt  # 会社名と会社コードをスペースで区切って組み合わせ
    encoded_query = urllib.parse.quote(query_string)  # 組み合わせた文字列をURLエンコード
    return base_url + encoded_query


financial_statements_dict = { "FYFinancialStatements_Consolidated_JP": "決算短信 (連結・日本基準)", "FYFinancialStatements_Consolidated_US": "決算短信 (連結・米国基準)", "FYFinancialStatements_NonConsolidated_JP": "決算短信 (非連結・日本基準)", "1QFinancialStatements_Consolidated_JP": "第1四半期短信 (連結・日本基準)", "1QFinancialStatements_Consolidated_US": "第1四半期短信 (連結・米国基準)", "1QFinancialStatements_NonConsolidated_JP": "第1四半期短信 (非連結・日本基準)", "2QFinancialStatements_Consolidated_JP": "第2四半期短信 (連結・日本基準)", "2QFinancialStatements_Consolidated_US": "第2四半期短信 (連結・米国基準)", "2QFinancialStatements_NonConsolidated_JP": "第2四半期短信 (非連結・日本基準)", "3QFinancialStatements_Consolidated_JP": "第3四半期短信 (連結・日本基準)", "3QFinancialStatements_Consolidated_US": "第3四半期短信 (連結・米国基準)", "3QFinancialStatements_NonConsolidated_JP": "第3四半期短信 (非連結・日本基準)", "OtherPeriodFinancialStatements_Consolidated_JP": "その他四半期短信 (連結・日本基準)", "OtherPeriodFinancialStatements_Consolidated_US": "その他四半期短信 (連結・米国基準)", "OtherPeriodFinancialStatements_NonConsolidated_JP": "その他四半期短信 (非連結・日本基準)", "FYFinancialStatements_Consolidated_JMIS": "決算短信 (連結・JMIS)", "1QFinancialStatements_Consolidated_JMIS": "第1四半期短信 (連結・JMIS)", "2QFinancialStatements_Consolidated_JMIS": "第2四半期短信 (連結・JMIS)", "3QFinancialStatements_Consolidated_JMIS": "第3四半期短信 (連結・JMIS)", "OtherPeriodFinancialStatements_Consolidated_JMIS": "その他四半期短信 (連結・JMIS)", "FYFinancialStatements_Consolidated_IFRS": "決算短信 (連結・IFRS)", "1QFinancialStatements_Consolidated_IFRS": "第1四半期短信 (連結・IFRS)", "2QFinancialStatements_Consolidated_IFRS": "第2四半期短信 (連結・IFRS)", "3QFinancialStatements_Consolidated_IFRS": "第3四半期短信 (連結・IFRS)", "OtherPeriodFinancialStatements_Consolidated_IFRS": "その他四半期短信 (連結・IFRS)", "FYFinancialStatements_NonConsolidated_Foreign": "決算短信 (非連結・外国基準)", "1QFinancialStatements_NonConsolidated_Foreign": "第1四半期短信 (非連結・外国基準)", "2QFinancialStatements_NonConsolidated_Foreign": "第2四半期短信 (非連結・外国基準)", "3QFinancialStatements_NonConsolidated_Foreign": "第3四半期短信 (非連結・外国基準)", "OtherPeriodFinancialStatements_NonConsolidated_Foreign": "その他四半期短信 (非連結・外国基準)", "FYFinancialStatements_Consolidated_Foreign": "決算短信 (連結・外国基準)", "1QFinancialStatements_Consolidated_Foreign": "第1四半期短信 (連結・外国基準)", "2QFinancialStatements_Consolidated_Foreign": "第2四半期短信 (連結・外国基準)", "3QFinancialStatements_Consolidated_Foreign": "第3四半期短信 (連結・外国基準)", "OtherPeriodFinancialStatements_Consolidated_Foreign": "その他四半期短信 (連結・外国基準)", "FYFinancialStatements_Consolidated_REIT": "決算短信 (REIT)", "DividendForecastRevision": "配当予想の修正", "EarnForecastRevision": "業績予想の修正", "REITDividendForecastRevision": "分配予想の修正", "REITEarnForecastRevision": "利益予想の修正" }
financial_statements_series = pd.Series(financial_statements_dict)

filtered_result = result[(start <= result.index) & (result.index <= end)].sort_values(by='pred_high_20', ascending=False)
filtered_result = filtered_result.reset_index()

# 最新の財務情報を持つデータフレームの準備
latest_fin = stock_fin_load.sort_values('DisclosedDate').drop_duplicates('LocalCode', keep='last')

# 必要な列のみを選択
latest_fin_dates = latest_fin[['LocalCode', 'TypeOfDocument', 'OperatingProfit', 'OrdinaryProfit', 'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'DisclosedDate', 'DisclosedTime',]]
stock_info = stock_list_load[["Code", "CompanyName", "Sector33CodeName", "Sector17CodeName", "MarketCodeName"]]

# filtered_result に最新の財務情報の日付をマージ
filtered_result = pd.merge(filtered_result, stock_info, how='left', left_on='code', right_on='Code')
filtered_result = pd.merge(filtered_result, latest_fin_dates, how='left', left_on='code', right_on='LocalCode')
filtered_result['DocumentTypeJP'] = filtered_result['TypeOfDocument'].map(financial_statements_series)
filtered_result['BingSearchUrl1'] = filtered_result.apply(lambda x: create_bing_search_url(x['CompanyName'], x['code'], prompt1), axis=1)
filtered_result['BingSearchUrl2'] = filtered_result.apply(lambda x: create_bing_search_url(x['CompanyName'], x['code'], prompt2), axis=1)
filtered_result['GoogleSearchUrl'] = filtered_result.apply(lambda x: create_google_search_url(x['CompanyName'], x['code']), axis=1)

filtered_result = filtered_result.drop(columns=['Code', 'LocalCode'])

filtered_result.to_csv(os.path.join(RESULT_DIR_PATH, "predict_result.csv"), index=False)

filtered_result

,datetime,code,pred_high_20,pred_low_20,CompanyName,Sector33CodeName,Sector17CodeName,MarketCodeName,TypeOfDocument,OperatingProfit,OrdinaryProfit,ForecastOperatingProfit,ForecastOrdinaryProfit,DisclosedDate,DisclosedTime,DocumentTypeJP,BingSearchUrl1,BingSearchUrl2,GoogleSearchUrl
0,2024-02-27,6862,0.138060,-0.141800,ミナトホールディングス,電気機器,電機・精密,スタンダード,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:00:00,配当予想の修正,https://www.bing.com/search?q=%E3%83%9F%E3%83%...,https://www.bing.com/search?q=%E3%83%9F%E3%83%...,https://www.google.com/search?q=%E3%83%9F%E3%8...
1,2024-02-27,2708,0.120025,-0.151793,久世,卸売業,商社・卸売,スタンダード,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:00:00,配当予想の修正,https://www.bing.com/search?q=%E4%B9%85%E4%B8%...,https://www.bing.com/search?q=%E4%B9%85%E4%B8%...,https://www.google.com/search?q=%E4%B9%85%E4%B...
2,2024-02-27,7901,0.113703,-0.138726,マツモト,その他製品,情報通信・サービスその他,スタンダード,EarnForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:30:00,業績予想の修正,https://www.bing.com/search?q=%E3%83%9E%E3%83%...,https://www.bing.com/search?q=%E3%83%9E%E3%83%...,https://www.google.com/search?q=%E3%83%9E%E3%8...
3,2024-02-27,4811,0.111123,-0.178071,ドリーム・アーツ,情報･通信業,情報通信・サービスその他,グロース,FYFinancialStatements_Consolidated_JP,5.770000e+08,5.630000e+08,NaN,NaN,2024-02-27,15:30:00,決算短信 (連結・日本基準),https://www.bing.com/search?q=%E3%83%89%E3%83%...,https://www.bing.com/search?q=%E3%83%89%E3%83%...,https://www.google.com/search?q=%E3%83%89%E3%8...
4,2024-02-27,7363,0.097370,-0.150267,ベビーカレンダー,サービス業,情報通信・サービスその他,グロース,FYFinancialStatements_NonConsolidated_JP,-3.100000e+07,-2.600000e+07,NaN,NaN,2024-02-27,16:30:00,決算短信 (非連結・日本基準),https://www.bing.com/search?q=%E3%83%99%E3%83%...,https://www.bing.com/search?q=%E3%83%99%E3%83%...,https://www.google.com/search?q=%E3%83%99%E3%8...
5,2024-02-27,4687,0.089388,-0.097850,ＴＤＣソフト,情報･通信業,情報通信・サービスその他,プライム,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:30:00,配当予想の修正,https://www.bing.com/search?q=%EF%BC%B4%EF%BC%...,https://www.bing.com/search?q=%EF%BC%B4%EF%BC%...,https://www.google.com/search?q=%EF%BC%B4%EF%B...
6,2024-02-27,5571,0.079389,-0.079000,エキサイトホールディングス,情報･通信業,情報通信・サービスその他,スタンダード,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:00:00,配当予想の修正,https://www.bing.com/search?q=%E3%82%A8%E3%82%...,https://www.bing.com/search?q=%E3%82%A8%E3%82%...,https://www.google.com/search?q=%E3%82%A8%E3%8...
7,2024-02-27,3666,0.072209,-0.111321,テクノスジャパン,情報･通信業,情報通信・サービスその他,スタンダード,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,16:10:00,配当予想の修正,https://www.bing.com/search?q=%E3%83%86%E3%82%...,https://www.bing.com/search?q=%E3%83%86%E3%82%...,https://www.google.com/search?q=%E3%83%86%E3%8...
8,2024-02-27,6508,0.064338,-0.067911,明電舎,電気機器,電機・精密,プライム,DividendForecastRevision,NaN,NaN,NaN,NaN,2024-02-27,15:00:00,配当予想の修正,https://www.bing.com/search?q=%E6%98%8E%E9%9B%...,https://www.bing.com/search?q=%E6%98%8E%E9%9B%...,https://www.google.com/search?q=%E6%98%8E%E9%9...
9,2024-02-27,7525,0.048765,-0.121200,リックス,卸売業,商社・卸売,プライム,3QFinancialStatements_Consolidated_JP,2.795000e+09,3.149000e+09,3.580000e+09,3.950000e+09,2024-02-27,14:00:00,第3四半期短信 (連結・日本基準),https://www.bing.com/search?q=%E3%83%AA%E3%83%...,https://www.bing.com/search?q=%E3%83%AA%E3%83%...,https://www.google.com/search?q=%E3%83%AA%E3%8...


# 通知

In [ ]:
import requests

class PushoverClient:
    def __init__(self):
        self.user_key = self.read_token("/content/drive/MyDrive/drive_ws/tokens/pushover_user_key")
        self.api_token = self.read_token("/content/drive/MyDrive/drive_ws/tokens/pushover_api_key")

    def read_token(self, file_path):
        try:
            with open(file_path, 'r') as file:
                return file.read().strip()
        except IOError:
            raise Exception(f"Unable to read token file: {file_path}")

    def send_message(self, message, title=None, url=None, url_title=None, priority=None, sound=None):
        data = {
            "token": self.api_token,
            "user": self.user_key,
            "message": message,
        }

        # オプショナルなパラメータの追加
        if title:
            data["title"] = title
        if url:
            data["url"] = url
        if url_title:
            data["url_title"] = url_title
        if priority:
            data["priority"] = priority
        if sound:
            data["sound"] = sound

        # Pushover APIへのリクエスト送信
        response = requests.post("https://api.pushover.net/1/messages.json", data=data)
        return response.json()

client = PushoverClient()
response = client.send_message("Hello, this is a test message from Python!", title="Test Message")
print(response)


{'status': 1, 'request': '88be675b-26ca-4857-9368-177e8d0b2ba2'}


# メモ



## 定期実行のタイミング
19時30分頃に回すのが良さそう

|   データの種類  |  更新タイミング  |
|    ---     |    ---     |
|  株価情報  |  日次で18時半頃に更新されます。  |
|  財務情報  |  当日分の速報を18時半頃、当日分の確報を翌日の0時半頃に更新されます。  |

https://jpx.gitbook.io/j-quants-api/api-reference/data-spec



## 推論のコツ

hiohiohioさんのアドバイス
https://discord.com/channels/1001344948597178409/1024255489380462602/1024592765109157928

hiohiohioのコード（８位）
https://github.com/J-Quants/jquants-api-client-python/blob/main/examples/20230126-006-mkdeco-jquants-kaggle-8th.ipynb